# Data

We obtained our dataset from New York City Gov [website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). For this project, we decided to use the Yellow Taxi Trip Records data for the year 2023. The data was available in parquet files for each month. All the data was donwloaded and can be found inside `/home/mjain3/uci150/mjain3/yellow_taxi_data`

Additionally, we also used New York shape data (from the same gov website mentioned above) to be able to do visualizations on the map using geopandas. The link to the shapefile is [here](https://d37ci6vzurychx.cloudfront.net/misc/taxi_zones.zip)

# Initialization

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import dayofmonth, month, hour, minute, year, col, floor, count, isnull, udf
from pyspark.sql.types import LongType, DecimalType, IntegerType, StructType, StructField, StringType, BooleanType

from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

from functools import reduce

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

import matplotlib.pyplot as plt

/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
spark = SparkSession.builder \
	.appName("Yellow Taxi Task 1") \
	.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/05 07:10:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# spark = SparkSession.builder \
#     .config("spark.driver.memory", "4g") \
# 	.config("spark.executor.memory", "2g") \
#     .config('spark.executor.instances', 6) \
# 	.appName("Yellow Taxi Task 1") \
# 	.getOrCreate()

In [4]:
spark

# Importing Data

In [5]:
base_path = 'yellow_taxi_data/yellow_tripdata_2023-{:02d}.parquet'

paths=[]
for mo in range(1, 2):  # This loops from 1 to 12
    path = base_path.format(mo)  # Formats the month with leading zero if necessary
    paths.append(path)

# paths = ['yellow_taxi_data/yellow_tripdata_2023-01.parquet', 'yellow_taxi_data/yellow_tripdata_2023-02.parquet']

In [6]:
# Function to load and cast a single Parquet file
def load_and_cast(filepath):
    df = spark.read.parquet(filepath)
    df = df.withColumn("VendorID", col("VendorID").cast(IntegerType()))
    return df

# Load, cast, and accumulate all DataFrames
dataframes = [load_and_cast(path) for path in paths]
df = reduce(lambda df1, df2: df1.unionByName(df2), dataframes)

# Show the DataFrame
df.show(1)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-01-01 00:32:10|  2023-01-01 00:40:36|            1.0|         0.97|       1.0|                 N|         161|         141|           2|        9.3|  1.0|    0.5|       0.

In [7]:
df.count()

3066766

In [8]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [9]:
print(df.rdd.getNumPartitions())

8


# Preprocessing

- Extraction of Time Components: extracted day, month, year, and time from the tpep_pickup_datetime column to separate columns (pickup_day, pickup_month, pickup_year).
- Labeling Holidays: used the USFederalHolidayCalendar to identify if a particular day is a holiday or not (is_holiday)
- 15-Minute, 30-Minute, and 60-Minute Indexing: calculated a 15/30/60min index for each record, representing which block of the day the pickup occurred. This was done by dividing the total minutes past midnight by 15/30/60.

In [10]:
# Filter selected features for the task: tpep_pickup_datetime and PULocationID 
df_task1 = df["tpep_pickup_datetime","PULocationID"]

# Add pickup: day, month, time columns to df
df_task1 = df_task1.withColumn("pickup_year", year("tpep_pickup_datetime"))
df_task1 = df_task1.withColumn("pickup_day", dayofmonth("tpep_pickup_datetime"))
df_task1 = df_task1.withColumn("pickup_month", month("tpep_pickup_datetime"))
df_task1 = df_task1.withColumn("pickup_time", hour("tpep_pickup_datetime") * 60 + minute("tpep_pickup_datetime"))

# Show the modified DataFrame structure and some rows to verify
df_task1.printSchema()
df_task1.show()

root
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_time: integer (nullable = true)

+--------------------+------------+-----------+----------+------------+-----------+
|tpep_pickup_datetime|PULocationID|pickup_year|pickup_day|pickup_month|pickup_time|
+--------------------+------------+-----------+----------+------------+-----------+
| 2023-01-01 00:32:10|         161|       2023|         1|           1|         32|
| 2023-01-01 00:55:08|          43|       2023|         1|           1|         55|
| 2023-01-01 00:25:04|          48|       2023|         1|           1|         25|
| 2023-01-01 00:03:48|         138|       2023|         1|           1|          3|
| 2023-01-01 00:10:29|         107|       2023|         1|           1|         10|
| 2023-01-01 00:50:34|         161|       202

In [11]:
# Function to determine if a date is a holiday
def is_holiday(date):
    year, month, day = date.year, date.month, date.day
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=f'{year}-01-01', end=f'{year}-12-31').to_pydatetime()
    return pd.Timestamp(year, month, day) in holidays

# Register UDF
is_holiday_udf = udf(is_holiday, BooleanType())

# Apply UDF to create a new column 'is_holiday'
df_task1 = df_task1.withColumn("is_holiday", is_holiday_udf(col("tpep_pickup_datetime")))

# Show the modified DataFrame structure and some rows to verify
df_task1.printSchema()
df_task1.show()

root
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- is_holiday: boolean (nullable = true)



+--------------------+------------+-----------+----------+------------+-----------+----------+
|tpep_pickup_datetime|PULocationID|pickup_year|pickup_day|pickup_month|pickup_time|is_holiday|
+--------------------+------------+-----------+----------+------------+-----------+----------+
| 2023-01-01 00:32:10|         161|       2023|         1|           1|         32|     false|
| 2023-01-01 00:55:08|          43|       2023|         1|           1|         55|     false|
| 2023-01-01 00:25:04|          48|       2023|         1|           1|         25|     false|
| 2023-01-01 00:03:48|         138|       2023|         1|           1|          3|     false|
| 2023-01-01 00:10:29|         107|       2023|         1|           1|         10|     false|
| 2023-01-01 00:50:34|         161|       2023|         1|           1|         50|     false|
| 2023-01-01 00:09:22|         239|       2023|         1|           1|          9|     false|
| 2023-01-01 00:27:12|         142|       2023|   

In [12]:
# Create 15 min index
df_task1 = df_task1.withColumn("15min_index", floor((hour(col("tpep_pickup_datetime")) * 60 + minute(col("tpep_pickup_datetime"))) / 15))
df_task1 = df_task1.withColumn("30min_index", floor((hour(col("tpep_pickup_datetime")) * 60 + minute(col("tpep_pickup_datetime"))) / 30))
df_task1 = df_task1.withColumn("60min_index", floor((hour(col("tpep_pickup_datetime")) * 60 + minute(col("tpep_pickup_datetime"))) / 60))

# Show the modified DataFrame structure and some rows to verify
df_task1.printSchema()
df_task1.show()

root
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- pickup_year: integer (nullable = true)
 |-- pickup_day: integer (nullable = true)
 |-- pickup_month: integer (nullable = true)
 |-- pickup_time: integer (nullable = true)
 |-- is_holiday: boolean (nullable = true)
 |-- 15min_index: long (nullable = true)
 |-- 30min_index: long (nullable = true)
 |-- 60min_index: long (nullable = true)

+--------------------+------------+-----------+----------+------------+-----------+----------+-----------+-----------+-----------+
|tpep_pickup_datetime|PULocationID|pickup_year|pickup_day|pickup_month|pickup_time|is_holiday|15min_index|30min_index|60min_index|
+--------------------+------------+-----------+----------+------------+-----------+----------+-----------+-----------+-----------+
| 2023-01-01 00:32:10|         161|       2023|         1|           1|         32|     false|          2|          1|          0|
| 2023-01-01 00:55:08|     

In [13]:
# Group by PULocationID and 60min_index, then count the occurrences
df_with_num_taxis_60min = df_task1.groupBy("PULocationID", "60min_index") \
                                  .agg(count("*").alias("num_taxis_60min"))

# Group by PULocationID and 30min_index, then count the occurrences
df_with_num_taxis_30min = df_task1.groupBy("PULocationID", "30min_index") \
                                  .agg(count("*").alias("num_taxis_30min"))

# Group by PULocationID and 15min_index, then count the occurrences
df_with_num_taxis_15min = df_task1.groupBy("PULocationID", "15min_index") \
                                  .agg(count("*").alias("num_taxis_15min"))

# Join the original DataFrame with the counts for 60min_index
df_final = df_task1.join(df_with_num_taxis_60min, on=["PULocationID", "60min_index"], how="left")

# Join the resulting DataFrame with the counts for 30min_index
df_final = df_final.join(df_with_num_taxis_30min, on=["PULocationID", "30min_index"], how="left")

# Join the resulting DataFrame with the counts for 15min_index
df_final = df_final.join(df_with_num_taxis_15min, on=["PULocationID", "15min_index"], how="left")

# Select the desired columns, including the 60min_index, 30min_index, and 15min_index
df_final = df_final.select("tpep_pickup_datetime", "PULocationID", "pickup_year", "pickup_day", 
                           "pickup_month", "pickup_time", "is_holiday", "60min_index", 
                           "30min_index", "15min_index", "num_taxis_60min", 
                           "num_taxis_30min", "num_taxis_15min")

# Show the result
df_final.show()

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=30047Kb max_used=30058Kb free=101024Kb
 bounds [0x00000001069e0000, 0x0000000108770000, 0x000000010e9e0000]
 total_blobs=11531 nmethods=10526 adapters=917
 compilation: disabled (not enough contiguous free space left)
+--------------------+------------+-----------+----------+------------+-----------+----------+-----------+-----------+-----------+---------------+---------------+---------------+
|tpep_pickup_datetime|PULocationID|pickup_year|pickup_day|pickup_month|pickup_time|is_holiday|60min_index|30min_index|15min_index|num_taxis_60min|num_taxis_30min|num_taxis_15min|
+--------------------+------------+-----------+----------+------------+-----------+----------+-----------+-----------+-----------+---------------+---------------+---------------+
| 2023-01-01 00:32:10|         161|       2023|         1|           1|         32|     false|          0|          1|          2|           1816|            750|            427|
| 2023-01-01 00:55:08|          43| 

In [14]:
df_final.cache()

DataFrame[tpep_pickup_datetime: timestamp_ntz, PULocationID: bigint, pickup_year: int, pickup_day: int, pickup_month: int, pickup_time: int, is_holiday: boolean, 60min_index: bigint, 30min_index: bigint, 15min_index: bigint, num_taxis_60min: bigint, num_taxis_30min: bigint, num_taxis_15min: bigint]

## Visualization to confirm pre-processing data

In [15]:
# # Convert to pd (For histogram Visualization)
# hist_data_15min = df_final.groupBy("15min_index").count().orderBy("15min_index")
# hist_data_30min = df_final.groupBy("30min_index").count().orderBy("30min_index")
# hist_data_60min = df_final.groupBy("60min_index").count().orderBy("60min_index")

# # hist_data_15min = df_task1.groupBy("15min_index").count().orderBy("15min_index")
# # hist_data_30min = df_task1.groupBy("30min_index").count().orderBy("30min_index")
# # hist_data_60min = df_task1.groupBy("60min_index").count().orderBy("60min_index")

# hist_pd_15min = hist_data_15min.toPandas()
# hist_pd_30min = hist_data_30min.toPandas()
# hist_pd_60min = hist_data_60min.toPandas()

# # Create a figure and a set of subplots
# fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))

# # Plotting on the first axis for 15min_index
# ax[0,0].bar(hist_pd_15min['15min_index'], hist_pd_15min['count'])
# ax[0,0].set_xlabel('15 Minute Index')
# ax[0,0].set_ylabel('Count')
# ax[0,0].set_title('Histogram of 15min_index')
# ax[0,0].set_xticks(range(0, 96, 5))

# # Plotting on the second axis for 30min_index
# ax[0,1].bar(hist_pd_30min['30min_index'], hist_pd_30min['count'], color='red')
# ax[0,1].set_xlabel('30 Minute Index')
# ax[0,1].set_ylabel('Count')
# ax[0,1].set_title('Histogram of 30min_index')
# ax[0,1].set_xticks(range(0, 48, 2))

# # Plotting on the second axis for 60min_index
# ax[1,0].bar(hist_pd_60min['60min_index'], hist_pd_60min['count'], color='green')
# ax[1,0].set_xlabel('60 Minute Index')
# ax[1,0].set_ylabel('Count')
# ax[1,0].set_title('Histogram of 60min_index')
# ax[1,0].set_xticks(range(0, 24, 1))

# # Display the plot
# plt.tight_layout()
# plt.show()

## Create the ground truth features

In [16]:
# Create ground truth columns for 15min, 30min, and 60min slots
# df_15min = df_task1.groupBy("pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "15min_index").agg(count("*").alias("num_taxis_15min"))
# df_30min = df_task1.groupBy("pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "30min_index").agg(count("*").alias("num_taxis_30min"))
# df_60min = df_task1.groupBy("pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "60min_index").agg(count("*").alias("num_taxis_60min"))

In [17]:
# df_60min.show(10)

In [18]:
# Join original DataFrame with the aggregated data
# df_15min = df_task1.join(df_15min, on=["pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "15min_index"], how="left")
# df_30min = df_task1.join(df_30min, on=["pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "30min_index"], how="left")
# df_60min = df_task1.join(df_60min, on=["pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "60min_index"], how="left")

# Vector Assembler

In [19]:
# feature_cols = ["pickup_year", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "pickup_time"]
feature_cols = ["60min_index", "pickup_month", "pickup_day", "PULocationID", "is_holiday", "pickup_time"]

# assembler_15min = VectorAssembler(inputCols=feature_cols, outputCol="features")
# assembler_30min = VectorAssembler(inputCols=feature_cols, outputCol="features")
assembler_60min = VectorAssembler(inputCols=feature_cols, outputCol="features")

# df_15min = assembler_15min.transform(df_15min)
# df_30min = assembler_30min.transform(df_30min)
df_60min = assembler_60min.transform(df_final)

In [20]:
# Summary statistics for num_taxis_15min
# df_15min.describe().show()

# Summary statistics for num_taxis_30min
# df_30min.describe().show()

# Summary statistics for num_taxis_60min
# df_60min.describe().show()

# Train-test split

In [21]:
# Split data into training and test sets
seed = 42

# train_15min, test_15min = df_15min.randomSplit([0.8, 0.2], seed=seed)
# train_30min, test_30min = df_30min.randomSplit([0.8, 0.2], seed=seed)
train_60min, test_60min = df_60min.randomSplit([0.8, 0.2], seed=seed)

# Model training

In [22]:
# evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="num_taxis_15min", metricName="rmse")
# evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="num_taxis_30min", metricName="rmse")
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="num_taxis_60min", metricName="rmse")

## XGBoost

In [23]:
from xgboost.spark import SparkXGBRegressor

In [24]:
xgboost_regressor = SparkXGBRegressor(features_col="features", label_col="num_taxis_60min", silent=False)
xgb_regression_model = xgboost_regressor.fit(train_60min)

2024-06-05 09:48:31,216 INFO XGBoost-PySpark: _fit Running xgboost-2.0.3 on 1 workers with
	booster params: {'objective': 'reg:squarederror', 'device': 'cpu', 'silent': False, 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[09:48:39] task 0 got new rank 0                                    (0 + 1) / 1]
/Users/mj/conda_env/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [09:48:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
2024-06-05 09:48:50,542 INFO XGBoost-PySpark: _fit Finished xgboost training!   


In [25]:
xgbr_predictions = xgb_regression_model.transform(test_60min)

In [26]:
xgbr_predictions

DataFrame[tpep_pickup_datetime: timestamp_ntz, PULocationID: bigint, pickup_year: int, pickup_day: int, pickup_month: int, pickup_time: int, is_holiday: boolean, 60min_index: bigint, 30min_index: bigint, 15min_index: bigint, num_taxis_60min: bigint, num_taxis_30min: bigint, num_taxis_15min: bigint, features: vector, prediction: double]

In [27]:
xgb_rmse_60min = evaluator.evaluate(xgbr_predictions)
print(f"RMSE for 60min model: {xgb_rmse_60min}")

INFO:XGBoost-PySpark:Do the inference on the CPUs
/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/Users/mj/conda_env/tf/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (dete

RMSE for 60min model: 96.65698779126816


In [31]:
# get feature importances for from xgboost.spark import SparkXGBRegressor
feature_importances = xgb_regression_model.getFea
print(feature_importances)

AttributeError: 'SparkXGBRegressorModel' object has no attribute 'getFeatureImportances'

## Linear Regression

In [32]:
# Define Linear Regression models
# lr_15min = LinearRegression(featuresCol="features", labelCol="num_taxis_15min")
# lr_30min = LinearRegression(featuresCol="features", labelCol="num_taxis_30min")
lr_60min = LinearRegression(featuresCol="features", labelCol="num_taxis_60min")

# Train the models
# lr_model_15min = lr_15min.fit(train_15min)
# lr_model_30min = lr_30min.fit(train_30min)
lr_model_60min = lr_60min.fit(train_60min)

24/06/05 10:01:43 WARN Instrumentation: [4ea08fd3] regParam is zero, which might cause numerical instability and overfitting.
24/06/05 10:01:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/06/05 10:01:49 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [34]:
# Make predictions
# lr_predictions_15min = model_lr_15min.transform(test_15min)
# lr_predictions_30min = model_lr_30min.transform(test_30min)
lr_predictions_60min = lr_model_60min.transform(test_60min)

In [36]:
# Evaluate models
# lr_rmse_15min = evaluator.evaluate(lr_predictions_15min)
# print(f"RMSE for 15min model: {lr_rmse_15min}")


# lr_rmse_30min = evaluator.evaluate(lr_predictions_30min)
# print(f"RMSE for 30min model: {lr_rmse_30min}")


lr_rmse_60min = evaluator.evaluate(lr_predictions_60min)
print(f"RMSE for LR 60min model: {lr_rmse_60min}")

RMSE for LR 60min model: 2817.1247046985304
